In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.11.0


In [4]:
import tensorboard
tensorboard.__version__

'2.11.0'

In [5]:
from keras.layers import Dense
from keras.models import Model
import pandas as pd
import datetime
import shutil
import numpy as np

In [6]:
class NeuNet(Model):
    def __init__(self, neurons=128):
        super(NeuNet, self).__init__()        
        self.in_layer = Dense(10, activation='relu')
        self.hidden_1 = Dense(neurons, activation='relu')
        self.hidden_2 = Dense(neurons, activation='relu')
        self.hidden_3 = Dense(neurons, activation='relu')
        self.hidden_4 = Dense(neurons, activation='relu')
        self.hidden_5 = Dense(neurons, activation='relu')
        self.hidden_6 = Dense(neurons, activation='relu')
        self.hidden_7 = Dense(neurons, activation='relu')
        self.hidden_8 = Dense(neurons, activation='relu')
        self.hidden_9 = Dense(neurons, activation='relu')
        self.hidden_10 = Dense(neurons, activation='relu')
        self.out_layer = Dense(1, activation='relu')

    def call(self, inputs):
        x = self.in_layer(inputs)
        x = self.hidden_1(x)
        x = self.hidden_2(x)
        x = self.hidden_3(x)
        x = self.hidden_4(x)
        x = self.hidden_5(x)
        x = self.hidden_6(x)
        x = self.hidden_7(x)
        x = self.hidden_8(x)
        x = self.hidden_9(x)
        x = self.hidden_10(x)
        outputs = self.out_layer(x)
        return outputs

In [7]:
def train_net(data: pd.DataFrame, labels: pd.DataFrame, net: NeuNet, loss: tf.keras.losses, optimizer: tf.keras.optimizers, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = net(data)
        loss = loss(labels, predictions)
    gradients = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)


def val_net(data: pd.DataFrame, labels: pd.DataFrame, net: NeuNet, loss: tf.keras.losses, val_loss, val_accuracy):
    predictions = net(data)
    loss = loss(labels, predictions)
    val_loss(loss)
    val_accuracy(labels, predictions)

In [8]:
BATCH_SIZE = 32
BUFFER_SIZE = 32
NEURONS = 128
LEARNING_RATE = 0.001
EPOCHS = 50

In [9]:
from pathlib import Path

In [10]:
in_dir = Path('data/prepared/')

X_train_name = in_dir / 'X_full.csv'
y_train_name = in_dir / 'y_full.csv'
X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_val_name = in_dir / 'X_val.csv'
y_val_name = in_dir / 'y_val.csv'
X_val = pd.read_csv(X_val_name)
y_val = pd.read_csv(y_val_name)

In [11]:
train_ds = tf.data.Dataset.from_tensor_slices(
(X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)

In [12]:
logs_path = Path('logs')
if logs_path.exists():
    shutil.rmtree(logs_path)

net = NeuNet(neurons=NEURONS)
net.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE), loss=tf.keras.losses.MeanSquaredError(), metrics=['mse'])


# train_loss = tf.keras.metrics.Mean(name='train_loss')
# train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_mse')
# val_loss = tf.keras.metrics.Mean(name='val_loss')
# val_accuracy = tf.keras.metrics.MeanSquaredError(name='val_mse')

# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
# val_log_dir = logs_path / 'gradient_tape' / current_time / 'val'
# train_log_dir.mkdir(exist_ok=True, parents=True)
# val_log_dir.mkdir(exist_ok=True, parents=True)
# train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
# val_summary_writer = tf.summary.create_file_writer(str(val_log_dir))

logdir = logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

net.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=tensorboard_callback)
                            
# for epoch in range(EPOCHS):
#     for (X_train, y_train) in train_ds:
#         with fit_summary_writer.as_default():
#             train_net(X_train, y_train, net, loss, optimizer, train_loss, train_accuracy)
#     with train_summary_writer.as_default():
#         tf.summary.scalar('loss', train_loss.result(), step=epoch)
#         tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
#     for (X_val, y_val) in val_ds:
#         val_net(X_val, y_val, net, loss, val_loss, val_accuracy)
#     with val_summary_writer.as_default():
#         tf.summary.scalar('loss', val_loss.result(), step=epoch)
#         tf.summary.scalar('mse', val_accuracy.result(), step=epoch)
#     template = 'Epoch {}, Loss: {}, Accuracy: {}, val Loss: {}, val MSE: {}'
#     print (template.format(epoch+1,
#                             train_loss.result(),
#                             train_accuracy.result(),
#                             val_loss.result(),
#                             val_accuracy.result()))
#     if epoch == EPOCHS - 1:
#         print("Loss: ", val_loss.result().numpy())
#     # Reset metrics every epoch
#     train_loss.reset_states()
#     val_loss.reset_states()
#     train_accuracy.reset_states()
#     val_accuracy.reset_states()
# with fit_summary_writer.as_default():
#     tf.summary.trace_export(
#         name="my_func_trace",
#         step=0,
#         profiler_outdir=logdir
#     )

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Epoch 1/50
230/230 [==============================] - 1s 2ms/step - loss: 18.4473 - mse: 18.4473
Epoch 2/50
230/230 [==============================] - 0s 2ms/step - loss: 0.3971 - mse: 0.3971
Epoch 3/50
230/230 [==============================] - 0s 1ms/step - loss: 0.3034 - mse: 0.3034
Epoch 4/50
230/230 [==============================] - 0s 1ms/step - loss: 0.2542 - mse: 0.2542
Epoch 5/50
230/230 [==============================] - 0s 1ms/step - loss: 0.2129 - mse: 0.2129
Epoch 6/50
230/230 [==============================] - 0s 1ms/step - loss: 0.1852 - mse: 0.1852
Epoch 7/50
230/230 [==============================] - 0s 1ms/step - loss: 0.1659 - mse: 0.1659
Epoch 8/50
230/230 [==============================] - 0s 1ms/step - loss: 0.1449 - mse: 0.1449
Epoch 9/50
230/230 [==============================] - 0s 1ms/step - loss: 0.1318 - mse: 0.1318
Epoch 10/50
230/230 [==============================] - 0s 1ms/step - l

In [13]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 17904), started 2 days, 18:10:50 ago. (Use '!kill 17904' to kill it.)